# ZYX Euler Angle Sequence (orientation and angular velocity)

Do imports.

In [1]:
import numpy as np
from scipy.spatial.transform import Rotation
import sympy as sym

## Compute orientation from angles

Define symbolic variables.

In [2]:
psi, theta, phi = sym.symbols('psi, theta, phi')

Define individual rotation matrices.

In [3]:
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi), sym.cos(psi), 0],
                 [0, 0, 1]])

Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)],
                 [0, 1, 0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])

Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi), sym.cos(phi)]])

Apply sequential transformation to compute the rotation matrix that describes the orientation $R^W_B$ of the drone.

In [4]:
R_inW_ofB = Rz @ Ry @ Rx

Show this rotation matrix.

In [ ]:
R_inW_ofB

Create a function that returns this rotation matrix as a numpy array.

In [6]:
get_R_inW_ofB = sym.lambdify((psi, theta, phi), R_inW_ofB)

Evaluate this function at particular angles (for example).

In [ ]:
print(get_R_inW_ofB(0.1, 0.2, 0.3))

Check that this result is the same as what we would have gotten using the `scipy.Rotation` package.

**NOTE.** This is how I found a mistake in my use of `scipy.Rotation` — compulsive checking like this is important!

In [ ]:
# Print the result so we can compare it by inspection
print(Rotation.from_euler('ZYX', [0.1, 0.2, 0.3]).as_matrix())

# Check the result is the same to some relative and absolute tolerance
assert(np.allclose(
    get_R_inW_ofB(0.1, 0.2, 0.3),
    Rotation.from_euler('ZYX', [0.1, 0.2, 0.3]).as_matrix(),
))

## Compute angular rates from angular velocity

Recall that

$$\begin{bmatrix} \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \end{bmatrix} = N w^W_{W, B}$$

for some matrix $N$. Here is how to compute that matrix for a ZYX Euler Angle sequence.  First, we compute its inverse.

In [9]:
Ninv = sym.Matrix.hstack((Ry * Rx).T * sym.Matrix([[0], [0], [1]]),
                              (Rx).T * sym.Matrix([[0], [1], [0]]),
                                       sym.Matrix([[1], [0], [0]]))

Then, we compute $N$ by taking the inverse of $N^{-1}$.

In [10]:
N = sym.simplify(Ninv.inv())

Show $N$ (remember that $\tan\theta = \sin\theta / \cos\theta$).

In [ ]:
N

Create a function that returns $N$ as a numpy array.

In [12]:
get_N = sym.lambdify((psi, theta, phi), N)

Do an example of finding angular rates from angular velocity.

In [ ]:
# Specify angular velocity
w_inB_ofWB = np.array([1., 2., 3.])

# Find angular rates for given angles and angular velocity
psi_dot, theta_dot, phi_dot = get_N(0.01, 0.02, 0.03) @ w_inB_ofWB

# Show results
print(f'ANGULAR VELOCITY: {w_inB_ofWB}')
print(f'ANGULAR RATES: {psi_dot:.2f}, {theta_dot:.2f}, {phi_dot:.2f}')